In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:

from keras.models import Sequential
from keras.layers import Dense,Conv2D,Input,MaxPooling2D,Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.constraints import maxnorm

In [4]:
def create_model(activ='relu',dp_rate=0.5,num_layers=1,kernals_per=1,opt='adam',kernal_shrink=2):
  num_kernals=int(kernals_per*64)
  model = Sequential()
  model.add(Input(shape=(28,28,1)))
  while num_layers:
    model.add(Conv2D(num_kernals, kernel_size=(3, 3), activation=activ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    num_layers=num_layers-1
    num_kernals=num_kernals*kernal_shrink  
  model.add(Flatten())
  model.add(Dropout(dp_rate))
  model.add(Dense(num_classes, activation="softmax"))
  return model

In [5]:
create_model().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10816)             0         
_________________________________________________________________
dropout (Dropout)            (None, 10816)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                108170    
Total params: 108,810
Trainable params: 108,810
Non-trainable params: 0
_________________________________________________________________


In [6]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [15]:
from sklearn import metrics
import tensorflow

In [29]:
def evaluate_network(dp_rate,lr,kernals_per,kernal_shrink):
        model = create_model(dp_rate=dp_rate, kernals_per=kernals_per,kernal_shrink=kernal_shrink)
        model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr))
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=100, verbose=0, mode='auto', restore_best_weights=True)

        # Train on the bootstrap sample
        model.fit(x_train,y_train,validation_data=(x_test,y_test),
                  callbacks=[monitor],verbose=0,epochs=10)
        #epochs = monitor.stopped_epoch
        #epochs_needed.append(epochs)

        # Predict on the out of boot (validation)
        pred = model.predict(x_test)

        # Measure this bootstrap's log loss
        y_compare = np.argmax(y_test,axis=1) # For log loss calculation
        score = metrics.log_loss(y_compare, pred)
        #mean_benchmark.append(score)
        #m1 = statistics.mean(mean_benchmark)
        #m2 = statistics.mean(epochs_needed)
        #mdev = statistics.pstdev(mean_benchmark)

        # Record this iteration
        #time_took = time.time() - start_time
        
        tensorflow.keras.backend.clear_session()
        return (-score)

In [17]:
print(evaluate_network(
    dp_rate=0.2,
    lr=1e-3,
    kernals_per=0.2,
    kernal_shrink=0.2)) 

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.5253 - val_loss: 0.1327
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1403 - val_loss: 0.0960
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1077 - val_loss: 0.0772
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0950 - val_loss: 0.0737
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0807 - val_loss: 0.0686
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0748 - val_loss: 0.0585
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0707 - val_loss: 0.0614
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0599 - val_loss: 0.0565
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0553 - val_loss: 0.0577
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.053

In [21]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=a6cd1ca274aa44f79d02f9d86824193238c74b0ac4831e3127c652191c8695a6
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [30]:
from bayes_opt import BayesianOptimization
import time

# Supress NaN warnings
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

# Bounded region of parameter space
pbounds = {'dp_rate': (0.0, 0.499),
           'lr': (0.0, 0.1),
           'kernals_per': (0.5, 1),
           'kernal_shrink': (1.5, 2)
          }

optimizer = BayesianOptimization(
    f=evaluate_network,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum 
    # is observed, verbose = 0 is silent
    random_state=1,
)


In [32]:
start_time = time.time()
optimizer.maximize(init_points=10, n_iter=10,)
time_took = time.time() - start_time

print(f"Total runtime: {hms_string(time_took)}")
print(optimizer.max)

|   iter    |  target   |  dp_rate  | kernal... | kernal... |    lr     |
-------------------------------------------------------------------------
|  1        | -0.1597   |  0.07323  |  1.546    |  0.5931   |  0.03456  |
|  2        | -0.313    |  0.198    |  1.769    |  0.7096   |  0.06852  |
|  3        | -0.4946   |  0.102    |  1.939    |  0.5137   |  0.06705  |
|  4        | -0.1148   |  0.2082   |  1.779    |  0.5702   |  0.01981  |
|  5        | -0.3985   |  0.3996   |  1.984    |  0.6567   |  0.06923  |
|  6        | -0.05539  |  0.4373   |  1.947    |  0.5425   |  0.003905 |
|  7        | -0.321    |  0.08475  |  1.939    |  0.5492   |  0.04211  |
|  8        | -0.1599   |  0.478    |  1.767    |  0.8459   |  0.03155  |
|  9        | -0.4736   |  0.3426   |  1.917    |  0.5091   |  0.07501  |
|  10       | -2.321    |  0.4934   |  1.874    |  0.6402   |  0.07893  |
|  11       | -0.1634   |  0.05151  |  1.724    |  0.9543   |  0.02936  |
|  12       | -2.31     |  0.1436   | 

NameError: ignored

In [33]:
print(optimizer.max)

{'target': -0.05539396090865791, 'params': {'dp_rate': 0.4373181869957231, 'kernal_shrink': 1.9473033317519237, 'kernals_per': 0.542522105684889, 'lr': 0.0039054783232882364}}
